In [28]:
# The robot history file here (Path for demonstration)
robot_history_path = "/home/mlw0504/Projects/rise-gen-data/RIS-167-records/epoch_462/bot_e_460_ro_70.h5_history"
video_blend_file = "/home/mlw0504/Projects/rise-gen/data/blender/video_simple.blend"
data_dir = "/home/mlw0504/Projects/rise-gen/data/video"
output_name = "bot_167_462_70.blend"

# Number of record frames used to produce one keyframe in the video
frame_interval = 1

# Visualize mode
# mode = "simple_transparent"
mode = "voxels"

In [19]:
import os
import numpy as np

os.makedirs(data_dir, exist_ok=True)

def color_map(normalized_stress: np.ndarray):
    colors = np.zeros([normalized_stress.shape[0], 3], dtype=np.float32)
    cmap = plt.get_cmap("jet")
    for idx, stress in enumerate(normalized_stress):
        colors[idx] = cmap(stress - 1e-5)[:3]
    return colors

In [29]:
import bpy
from tqdm import tqdm
from visualize.blender.reader import RiseSimulationDataReader
from visualize.blender.generate import compute_transform_for_align_trajectory_to_y, generate_robot
from visualize.blender.scene import setup_scene

setup_scene(video_blend_file,
            use_gpu=True)

# Use the RS_SimulationDataReader to read the data
with RiseSimulationDataReader(robot_history_path) as reader:
    transform = compute_transform_for_align_trajectory_to_y(reader)
    
    # Create a new collection named "video" and link it to the scene
    video_collection_name = "video"
    if video_collection_name in bpy.data.collections:
        video_collection = bpy.data.collections[video_collection_name]
    else:
        video_collection = bpy.data.collections.new(video_collection_name)
        bpy.context.scene.collection.children.link(video_collection)

    # Loop through all frames
    frames = list(range(0, len(reader), frame_interval))
    real_frame_index = 0
    for frame_index in tqdm(frames):
        # Create a new collection for this frame
        frame_collection_name = f"frame_{frame_index}"
        frame_collection = bpy.data.collections.new(frame_collection_name)

        # Link the frame collection under the "video" collection
        video_collection.children.link(frame_collection)

        # Read voxel data for the current frame
        voxel_data = reader.read_voxel_data(frame_index)
        positions, orientations = voxel_data["position"], voxel_data["orientation"]
        transformed_positions, _ = transform(positions, orientations)
        transformed_com = np.mean(transformed_positions, axis=0)

        # Generate the robot in the scene
        robot_objects = generate_robot(
            frame_collection,
            reader,
            frame_index,
            color_map=color_map,
            transform=transform,
            mode=mode,
            voxel_size=0.01
        )

        # For each object in the robot, set visibility keyframes
        for obj in frame_collection.objects:
            # Initially hide the object
            obj.hide_viewport = True
            obj.hide_render = True

            # Insert keyframes for hiding the object
            obj.keyframe_insert(data_path="hide_viewport", frame=real_frame_index - 1)
            obj.keyframe_insert(data_path="hide_render", frame=real_frame_index - 1)

            # Show the object on the current frame
            obj.hide_viewport = False
            obj.hide_render = False

            # Insert keyframes for showing the object
            obj.keyframe_insert(data_path="hide_viewport", frame=real_frame_index)
            obj.keyframe_insert(data_path="hide_render", frame=real_frame_index)

            # Hide the object again on the next frame
            obj.hide_viewport = True
            obj.hide_render = True

            # Insert keyframes for hiding the object
            obj.keyframe_insert(data_path="hide_viewport", frame=real_frame_index + 1)
            obj.keyframe_insert(data_path="hide_render", frame=real_frame_index + 1)
        real_frame_index += 1

# Save the modified Blender file
bpy.ops.wm.save_as_mainfile(filepath=os.path.join(data_dir, output_name))


100%|██████████| 713/713 [06:10<00:00,  1.92it/s]


Read blend: "/home/mlw0504/Projects/rise-gen/data/blender/video_simple.blend"
regiondata free error
regiondata free error
regiondata free error
regiondata free error
Info: Total files 0 | Changed 0 | Failed 0
Info: Saved "bot_167_600_18.blend"


{'FINISHED'}